# Qiskit ExactReciprocal: Matrix for HHL

This notebook computes the Qiskit ExactReciprocal component on all basis states for certain numbers of clock qubits, 
from which we can determine the matrix form of the component.

For the HHL algorithm, as implemented in Jack Morgan's code, we use `neg_vals=True`, and also set `scaling` as $2\times 2^{-n}$, where $n$ is the number of clock qubits.

## See also:

* [Qiskit ExactReciprocal documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ExactReciprocal)
* [Jack Morgan's Enhanced HHL code](https://github.com/jackhmorgan/Enhanced-Hybrid-HHL)

In [11]:
import numpy as np
from IPython.display import display, Markdown
from math import log2
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import ExactReciprocal, StatePreparation
from qiskit.quantum_info import Statevector


def create_input_circuit(input):
    x = np.array(input) / np.linalg.norm(input)
    num_qubits = int(log2(x.size))
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_qubits, 'x')
    circuit = QuantumCircuit(x_reg)
    circuit.append(state_preparation, x_reg)
    return circuit


def create_exact_reciprocal_circuit(input, neg_vals=False):
    x = np.array(input) / np.linalg.norm(input)
    num_qubits = int(log2(x.size))
    scaling = 2**(1-num_qubits)
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_qubits, 'x')
    circuit = QuantumCircuit(x_reg)
    circuit.append(state_preparation, x_reg)
    exact_reciprocal = ExactReciprocal(num_state_qubits=num_qubits-1, scaling=scaling, neg_vals=neg_vals)
    circuit.append(exact_reciprocal, x_reg[::])
    return circuit


def latex_circuit_output(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    return simulator.run(aer_circuit).result().get_statevector().draw(output='latex').data.replace('|', r'\|')


def run_table(vals, neg_vals=False):
    markdowntable = f"""
| Input $x$ | Input State | Output $\\frac{{s}}{{x}}$ | Output State |
|-----------|-------------|-----------------------|--------------|
"""
    for val in vals:
        input = val["input"]
        input_vec = val["input_vec"]
        output = val["output"]
        input_state = latex_circuit_output(create_input_circuit(input_vec))
        output_state = latex_circuit_output(create_exact_reciprocal_circuit(input_vec, neg_vals))
        markdowntable += f"| {input} | {input_state} | {output} | {output_state} |\n"
    display(Markdown(markdowntable))


vals3 = [
    {"input": "$0$"           , "input_vec": [1, 0, 0, 0, 0, 0, 0, 0], "output": "undefined"},       # |0>|00>
    {"input": "$\\frac{1}{4}$", "input_vec": [0, 1, 0, 0, 0, 0, 0, 0], "output": "$1$"},             # |0>|01>
    {"input": "$\\frac{1}{2}$", "input_vec": [0, 0, 1, 0, 0, 0, 0, 0], "output": "$\\frac{1}{2}$"},  # |0>|10>
    {"input": "$\\frac{3}{4}$", "input_vec": [0, 0, 0, 1, 0, 0, 0, 0], "output": "$\\frac{1}{3}$"},  # |0>|11>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 1, 0, 0, 0], "output": ""},                # |1>|00>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 1, 0, 0], "output": ""},                # |1>|01>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 1, 0], "output": ""},                # |1>|10>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 1], "output": ""},                # |1>|11>
]

vals4 = [
    {"input": "$0$"           , "input_vec": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "undefined"},      # |0>|000>
    {"input": "$\\frac{1}{8}$", "input_vec": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$1$"},            # |0>|001>
    {"input": "$\\frac{1}{4}$", "input_vec": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{2}$"}, # |0>|010>
    {"input": "$\\frac{3}{8}$", "input_vec": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{3}$"}, # |0>|011>
    {"input": "$\\frac{1}{2}$", "input_vec": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{4}$"}, # |0>|100>
    {"input": "$\\frac{5}{8}$", "input_vec": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{5}$"}, # |0>|101>
    {"input": "$\\frac{3}{4}$", "input_vec": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{6}$"}, # |0>|110>
    {"input": "$\\frac{7}{8}$", "input_vec": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{7}$"}, # |0>|111>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], "output": ""},               # |1>|000>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], "output": ""},               # |1>|001>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], "output": ""},               # |1>|010>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], "output": ""},               # |1>|011>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], "output": ""},               # |1>|100>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], "output": ""},               # |1>|101>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], "output": ""},               # |1>|110>
    {"input": "(N/A)"         , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], "output": ""},               # |1>|111>
]

In [8]:
# num_state_qubits = 2; scaling = 1/4, neg_vals = False
run_table(vals3)


| Input $x$ | Input State | Output $\frac{s}{x}$ | Output State |
|-----------|-------------|-----------------------|--------------|
| $0$ | $$ \|000\rangle$$ | undefined | $$ \|000\rangle$$ |
| $\frac{1}{4}$ | $$ \|001\rangle$$ | $1$ | $$ \|101\rangle$$ |
| $\frac{1}{2}$ | $$ \|010\rangle$$ | $\frac{1}{2}$ | $$\frac{\sqrt{3}}{2} \|010\rangle+\frac{1}{2} \|110\rangle$$ |
| $\frac{3}{4}$ | $$ \|011\rangle$$ | $\frac{1}{3}$ | $$\frac{2 \sqrt{2}}{3} \|011\rangle+\frac{1}{3} \|111\rangle$$ |
| (N/A) | $$ \|100\rangle$$ |  | $$ \|100\rangle$$ |
| (N/A) | $$ \|101\rangle$$ |  | $$- \|001\rangle$$ |
| (N/A) | $$ \|110\rangle$$ |  | $$- \frac{1}{2} \|010\rangle+\frac{\sqrt{3}}{2} \|110\rangle$$ |
| (N/A) | $$ \|111\rangle$$ |  | $$- \frac{1}{3} \|011\rangle+\frac{2 \sqrt{2}}{3} \|111\rangle$$ |


In [12]:
# num_state_qubits = 3, scaling = 1/4, neg_vals = False
run_table(vals4)


| Input $x$ | Input State | Output $\frac{s}{x}$ | Output State |
|-----------|-------------|-----------------------|--------------|
| $0$ | $$ \|0000\rangle$$ | undefined | $$ \|0000\rangle$$ |
| $\frac{1}{8}$ | $$ \|0001\rangle$$ | $1$ | $$ \|1001\rangle$$ |
| $\frac{1}{4}$ | $$ \|0010\rangle$$ | $\frac{1}{2}$ | $$\frac{\sqrt{3}}{2} \|0010\rangle+\frac{1}{2} \|1010\rangle$$ |
| $\frac{3}{8}$ | $$ \|0011\rangle$$ | $\frac{1}{3}$ | $$\frac{2 \sqrt{2}}{3} \|0011\rangle+\frac{1}{3} \|1011\rangle$$ |
| $\frac{1}{2}$ | $$ \|0100\rangle$$ | $\frac{1}{4}$ | $$\frac{\sqrt{15}}{4} \|0100\rangle+\frac{1}{4} \|1100\rangle$$ |
| $\frac{5}{8}$ | $$ \|0101\rangle$$ | $\frac{1}{5}$ | $$0.9797958971 \|0101\rangle+\frac{1}{5} \|1101\rangle$$ |
| $\frac{3}{4}$ | $$ \|0110\rangle$$ | $\frac{1}{6}$ | $$0.9860132972 \|0110\rangle+\frac{1}{6} \|1110\rangle$$ |
| $\frac{7}{8}$ | $$ \|0111\rangle$$ | $\frac{1}{7}$ | $$0.9897433186 \|0111\rangle+\frac{1}{7} \|1111\rangle$$ |
| (N/A) | $$ \|1000\rangle$$ |  | $$ \|1000\rangle$$ |
| (N/A) | $$ \|1001\rangle$$ |  | $$- \|0001\rangle$$ |
| (N/A) | $$ \|1010\rangle$$ |  | $$- \frac{1}{2} \|0010\rangle+\frac{\sqrt{3}}{2} \|1010\rangle$$ |
| (N/A) | $$ \|1011\rangle$$ |  | $$- \frac{1}{3} \|0011\rangle+\frac{2 \sqrt{2}}{3} \|1011\rangle$$ |
| (N/A) | $$ \|1100\rangle$$ |  | $$- \frac{1}{4} \|0100\rangle+\frac{\sqrt{15}}{4} \|1100\rangle$$ |
| (N/A) | $$ \|1101\rangle$$ |  | $$- \frac{1}{5} \|0101\rangle+0.9797958971 \|1101\rangle$$ |
| (N/A) | $$ \|1110\rangle$$ |  | $$- \frac{1}{6} \|0110\rangle+0.9860132972 \|1110\rangle$$ |
| (N/A) | $$ \|1111\rangle$$ |  | $$- \frac{1}{7} \|0111\rangle+0.9897433186 \|1111\rangle$$ |
